In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import tweepy
import re
import time

In [17]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [4]:
article = soup.find("li", class_='slide')
title = article.find("div", class_="content_title").text
teaser = article.find("div", class_ ="article_teaser_body").text
print(title)
print(teaser)

NASA Readies Perseverance Mars Rover's Earthly Twin 
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


In [5]:
imageSiteUrl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(imageSiteUrl)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [7]:
imageEnd = soup.find("a", class_="button fancybox")["data-fancybox-href"]
featuredImage = "https://www.jpl.nasa.gov" + imageEnd
print(featuredImage)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17652_ip.jpg


In [8]:
marsUrl = "https://twitter.com/marswxreport?lang=en"
browser.visit(marsUrl)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [9]:
tweets = soup.find_all('div', class_='css-1dbjc4n')
for tweet in tweets:
    marsWeather = tweet.find('div', class_ = 'r-bnwqim').find('span').text
    if marsWeather:
        break
print(marsWeather)

InSight sol 636 (2020-09-10) low -94.6ºC (-138.2ºF) high -17.1ºC (1.2ºF)
winds from the W at 7.3 m/s (16.4 mph) gusting to 20.9 m/s (46.8 mph)
pressure at 7.80 hPa


In [10]:
factsUrl = 'https://space-facts.com/mars/'
facts = pd.read_html(factsUrl)
facts

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [14]:
factsDF = facts[0]
factsDF.columns = ['Description','Value']
factsDF.set_index('Description', inplace=True)
factsDF.head()

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [15]:
factsHtml = factsDF.to_html()

In [22]:
hemosphereUrl = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemosphereUrl)
time.sleep(1)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [23]:
hemispheres = soup.find_all('div', class_ = 'description')
hemiHolder = []
baseUrl = 'https://astrogeology.usgs.gov'

for hemisphere in hemispheres:
    title = hemisphere.find('h3').text
    urlEnd = baseUrl + hemisphere.find('a', class_ = "itemLink product-item")["href"]
    
    browser.visit(urlEnd)
    currentUrl = browser.html
    soup = BeautifulSoup(currentUrl, "html.parser")
    
    fullUrl = baseUrl + soup.find('img', class_ = "wide-image")["src"]
    
    hemiHolder.append({'title': title, "img": fullUrl})
hemiHolder

[{'title': 'Cerberus Hemisphere Enhanced',
  'img': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]